# 機械学習入門_AIプロジェクト演習

## 前提（作業環境）

データベースはPostgres10、機械学習環境はAnaconda Python3系にて事前の環境構築をお願いします。<br>
DB環境は使い慣れたものがあれば代替していただいても構いませんが、回答コード例は当該環境用のもののみの準備となります。
- Python3系のAnaconda
- Postgres10（pgAdmin4を使用）

## 1. データの取り込みと集計用VIEWの作成

<b>[事前演習1.1]</b><br>クライアントからe-commerceの購買取引データを受領し、優良顧客リテンションのための予測モデル構築のフィージビリティ検証を行うことになった。「input」フォルダ内の「ec_trans.csv」データを、作業用に準備したデータベースへテーブル名「raw_ec_trans」で取り込み、データ件数を確認せよ。ただし、クライアントはテーブル定義書を有していないとのことで、データ型は一旦全てvarchar(50)として取り込むものとする。カラム構成と名称はCSVファイルの1行目で確認できる。

<b>[事前演習1.2]</b><br>データの中身を確認しクライアントに問い合わせたところ、以下の回答を得た。
- StockCodeの小文字と大文字の違いに意味はない
- StockCode5桁目までで1つの商品類型を表し6桁目以降は色やサイズ違いを表している
- InvoiceDateの日付は"MM/DD/YYYY"で並んでいる

上記を踏まえ、以下を満たす「v_ec_trans」という名称のViewを作成せよ。
- 生データのStockCodeをStockCode_orgと名称変更する
- 生データのStockCodeを全て大文字に変換したカラムを作成し、名称をStockCodeとする
- 大文字変換されたStockCodeの左5桁を、StockCode2というカラムとする
- カラムQuantityとUnitPriceのデータ型を、float型に変換する
- Invoicedateのデータをタイムスタンプに変換する

<b>[事前演習1.3]</b><br>受領したデータの取引期間（InvocieDateの最小値と最大値）を確認せよ。

## 2. データ確認とクレンジング（目安時間：20分）

<b>[演習2.1]</b><br>CustomerIDが非NULLのデータを対象に、データ件数、CustomerIDのユニーク件数、InvoiceNoのユニーク件数を確認せよ。

<b>[演習2.2]</b><br>CustomerIDが非NULLのデータを対象に、購買数量（Quantity）と単価（UnitPrice）の最小値・平均値・最大値を確認せよ。

<b>[演習2.3]</b><br>
購買数量（Quantity）が負となるケースは注文のキャンセルで生じることがわかった。<br>
キャンセルはInvocieNoが"C"で始まるという。<br>
CustomerIDが非Nullのデータを対象に、以下2つのケースについて数値確認せよ。
1. InvoiceNoが"C"始まりのときの、Qunatityの最小値・平均値・最大値
1. InvoiceNoが"C"始まりでないときの、Qunatityの最小値・平均値・最大値

<b>[確認しよう]</b><br>上記結果を踏まえ、「output」フォルダにある「AIジョブカレ_機械学習入門_初回報告書サンプル.pptx」の「1ページ」を確認しよう。<br>
※データを観察すること、マクロ数値を把握すること、データに関する早期の質問の大切さを意識しよう。

## 3. 予測用データマート設計に関する集計（目安時間：30分）

<b>[演習3.1]</b><br>
以下条件の集計クエリを作成しよう。
- クライアントによると現在、半年間における購買回数(InvoiceNoのユニーク数)が4回以上か未満かで、顧客ランクを管理しているという。
- 2011年1月から6月末までの半年間のデータを対象に、顧客ランク別に、顧客数、購買金額(Quantity*Unitprice)、購買回数を集計せよ。
- ただし、CustomerIDが非NULL、UnitPriceが正のデータを対象に行うこと。

<b>[確認しよう]</b><br>
上記集計結果と以下の内容を確認しよう。
- 「output」フォルダにある「作業用」エクセルファイルの「優良」シートの内容
- 「AIジョブカレ_機械学習入門_初回報告書サンプル.pptx」の「2ページ」

<b>[演習3.2]</b><br>
以下条件の集計クエリを作成しよう。
- 2010年12月1日～2011年5月末日の6か月間における優良顧客（購買回数4回以上の顧客）のうち、
- 2011年6月1日～2011年8月末日までの3ヵ月間に購買が1回以上ある人数（リピート人数）を集計せよ。
- 同様に、全顧客（購買回数1回以上）についてもリピート人数を集計せよ。
- ただし、集計はCustomerIDが非NULL、UnitPriceが正のデータを対象に行うこと。

<b>[確認しよう]</b><br>
上記集計結果と以下内容を確認し、離反率をスナップショットではなくトレンドで把握しよう。
- 作業用エクセルの「トレンド」シートの内容
- 「AIジョブカレ_機械学習入門_初回報告書サンプル.pptx」の「3ページ」
- 注意：課題設定上、離反トレンドの明確な上昇や、離反率が何かしらの基準（業界平均など）より高い証拠を本来示したいが、本演習では妥協した。

<b>[確認しよう]</b><br>
作業用エクセルのシート「カテゴリ別」の内容から、購買回数（trips）の多い「人気カテゴリ」を確認しよう。<br>
人気カテゴリは特徴量設計に活用する。

## 4.データマートの作成と予測モデルの構築評価（目安時間：45分）

<b>[演習4.1]</b><br>以下を確認し、モデル用データマート作成用のクエリを完成させ、CSVファイルで出力せよ。
- 「作業用」エクセルのシート「設計」で、各データマートの特徴量計算期間、正解ラベルの定義期間を確認せよ。
- 「作業用」エクセルのシート「特徴量」で、特徴量定義を確認せよ。
- CSVの最初の列をCustomerID、最終の列を正解ラベルとして出力せよ。

<b>[演習4.2]</b><br>
フォルダ「\work\python」内にある「classifier.py」を完成させよ。<br>
(対応箇所は計7箇所あります。"[-------]"の箇所を変更して下さい)。

<b>[確認しよう]</b><br>
- 上記`classifier.py`の実行結果（K-Fold法によるモデル評価結果）と、「作業用」エクセルの「モデル管理」シートの値を確認せよ。
- 「AIジョブカレ_機械学習入門_初回報告書サンプル.pptx」の「8ページ」を確認せよ。

## 5. フォーワード検証（目安時間：15分）

<b>[演習5]</b><br>演習4.2で、最もパフォーマンスの高かったモデルによって出力された予測スコアデータ(「_with_pred」で終わるCSVファイル)をDBに取り込み、スコアを降順に10分割し、当該ランクごとに以下を出力する集計クエリを完成させよ。
- スコアランク
- 予測スコアの最小値
- 予測スコアの平均値
- 予測スコアの最大値
- レコード数[A]
- ターゲット（正例）数[B]
- ターゲット（正例）含有率（[B]/[A])

<b>[確認しよう]</b><br>
- 上記集計結果と、作業用エクセルのシート「FWD検証」の内容を確認しよう。
- 「AIジョブカレ_機械学習入門_初回報告書サンプル.pptx」の「11ページ」を確認しよう。
- [注意1]上位ランクほどTGT含有率平均からのアップリスト(%)が高いことを期待されていることを意識しよう。
- [注意2]どの程度アップリフトがあればビジネス上有用かを指摘するのもデータサイエンティストの役割と認識しよう。